# Initial Setup

In [3]:
import sys
sys.path.insert(0, "..")

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/jethro/anaconda3/envs/urop/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jethro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jethro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jethro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jethro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Data Processing
In this notebook, we will be processing the data in the ACM Dataset.

The first step is to understand the information we want to extract from our dataset.

In [7]:
PATH="/diskA/jethro/acm/"

In [8]:
import glob
files = glob.glob(f'{PATH}/*.xml')
files[:5]

['/diskA/jethro/acm/PROC-PROMISE12-2012-2365324.xml',
 '/diskA/jethro/acm/PROC-SCOPES07-2007-1269843.xml',
 '/diskA/jethro/acm/PROC-WSC78-1978-800252.xml',
 '/diskA/jethro/acm/PROC-PMAM12-2012-2141702.xml',
 '/diskA/jethro/acm/PROC-SIGIR87-1987-42005.xml']

In [9]:
from urop.xml import ACMCorpus

texts = ACMCorpus(files)

2018-03-09 12:07:29,140 : INFO : 'pattern' package not found; tag filters are not available for English


In [10]:
from urop.nlp import clean_text
clean_texts = [clean_text(text) for text in texts]

2018-03-09 12:07:55,616 : INFO : Parsing /diskA/jethro/acm/PROC-PROMISE12-2012-2365324.xml
2018-03-09 12:07:55,678 : INFO : Parsing /diskA/jethro/acm/PROC-SCOPES07-2007-1269843.xml
2018-03-09 12:07:59,092 : INFO : Parsing /diskA/jethro/acm/PROC-WSC78-1978-800252.xml
2018-03-09 12:08:02,426 : INFO : Parsing /diskA/jethro/acm/PROC-PMAM12-2012-2141702.xml
2018-03-09 12:08:06,543 : INFO : Parsing /diskA/jethro/acm/PROC-SIGIR87-1987-42005.xml
2018-03-09 12:08:12,303 : INFO : Parsing /diskA/jethro/acm/PROC-AAMAS05-2005-1082473.xml
2018-03-09 12:08:45,893 : INFO : Parsing /diskA/jethro/acm/PROC-FORTH89-1989-73312.xml
2018-03-09 12:08:45,902 : INFO : Parsing /diskA/jethro/acm/PROC-SODA92-1992-139404.xml
2018-03-09 12:08:55,079 : INFO : Parsing /diskA/jethro/acm/PROC-SCCG07-2007-2614348.xml
2018-03-09 12:08:59,728 : INFO : Parsing /diskA/jethro/acm/PROC-ICSE01-2001-381473.xml
2018-03-09 12:09:12,979 : INFO : Parsing /diskA/jethro/acm/PROC-STOC79-1979-800135.xml
2018-03-09 12:09:18,968 : INFO : 

In [11]:
from gensim import corpora
dct = corpora.Dictionary(clean_texts)

2018-03-09 12:11:56,440 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-09 12:11:57,384 : INFO : built Dictionary(126956 unique tokens: ['absence', 'absorption', 'abstraction', 'access', 'account']...) from 2338 documents (total 2434525 corpus positions)


In [14]:
corpus = [dct.doc2bow(text) for text in clean_texts]

In [15]:
dct.filter_extremes(no_below=3, no_above=0.5)
dct.save(f'{PATH}/dictionary.pkl')

2018-03-09 12:12:29,302 : INFO : discarding 107849 tokens: [('analysis', 1179), ('application', 1390), ('approach', 1432), ('asearlyas', 1), ('aslateas', 1), ('backpressure', 2), ('bekooij', 2), ('bilsen', 2), ('businterface', 1), ('carinfotainment', 1)]...
2018-03-09 12:12:29,302 : INFO : keeping 19107 tokens which were in no less than 3 and no more than 1169 (=50.0%) documents
2018-03-09 12:12:29,330 : INFO : resulting dictionary: Dictionary(19107 unique tokens: ['absence', 'absorption', 'abstraction', 'access', 'account']...)
2018-03-09 12:12:29,336 : INFO : saving Dictionary object under /diskA/jethro/acm//dictionary.pkl, separately None
2018-03-09 12:12:29,341 : INFO : saved /diskA/jethro/acm//dictionary.pkl


In [17]:
from gensim.corpora.mmcorpus import MmCorpus
MmCorpus.serialize(f'{PATH}/corpus.mm', corpus)

2018-03-09 12:12:49,174 : INFO : storing corpus in Matrix Market format to /diskA/jethro/acm//corpus.mm
2018-03-09 12:12:49,175 : INFO : saving sparse matrix to /diskA/jethro/acm//corpus.mm
2018-03-09 12:12:49,176 : INFO : PROGRESS: saving document #0
2018-03-09 12:12:49,471 : INFO : PROGRESS: saving document #1000
2018-03-09 12:12:49,734 : INFO : PROGRESS: saving document #2000
2018-03-09 12:12:49,840 : INFO : saved 2338x126956 matrix, density=0.252% (747740/296823128)
2018-03-09 12:12:49,841 : INFO : saving MmCorpus index to /diskA/jethro/acm//corpus.mm.index
